In [ ]:
%run ../../Environment/environment.ipynb

# 1. Bibliotheken importieren

In [ ]:
import gym
import numpy as np
import pandas as pd
import joblib
import random
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.callbacks import CheckpointCallback
from sklearn.preprocessing import StandardScaler
from stable_baselines3.common.vec_env import DummyVecEnv


# 2. Seed setzen

In [ ]:
seed = 42
SEED  = seed % (2**32 - 1)
print(f"SEED: {SEED}")

# 3. Daten einlesen

In [ ]:
# -------------------------------
# CSV Datem einlesen
# -------------------------------
train_data = pd.read_csv("../../Transform_data/stand_data/2023-2018_stand_data.csv")
train_data.drop('datetime', axis=1, inplace=True)

test_data = pd.read_csv("../../Transform_data/stand_data/2025-2024_stand_data.csv")
test_data.drop('datetime', axis=1, inplace=True)

if(train_data is not None and test_data is not None):
    print("Daten erfolgreich eingelesen")

# 4. Umgebung erstellen

In [ ]:
# -------------------------------
# TradingEnv erstellen (bitte sicherstellen, dass TradingEnv importiert oder im gleichen Skript definiert ist)
# -------------------------------
env = TradingEnv(
    data=train_data,
    initial_cash=10_000,
    window_size=336,
    scaler_path="../../Transform_data/scaler.pkl",
    default_seed=SEED
)

if(env is not None):
    print("Environment created successfully")

# 5. Agent erstellen und trainieren

In [ ]:
# -------------------------------
# PPO-Agenten initialisieren
# -------------------------------
model = A2C(
    "MlpPolicy",
    env,  # Normalisiert die Vorteile (Advantages) für stabileres Lernen
    verbose=1
)

# Optional: Checkpoint Callback um den Trainingsfortschritt zwischendurch zu speichern
checkpoint_callback = CheckpointCallback(save_freq=100, save_path='./logs/', name_prefix='a2c_trading')

# -------------------------------
# Training
# -------------------------------
model.learn(
    total_timesteps=100000, 
    #callback=checkpoint_callback, 
    #log_interval=1
)

# Speichere das trainierte Modell
model.save("a2c_trading_model")